## Header


**Created**: 11 Feb 2024  

Vishnu.

Modification History: Created to do cell-by-cell merge of https://colab.research.google.com/github/CrashingGuru/FGAN-Build-a-thon/blob/main/Notebooks2023/Fine_tune_Mistral_7b_model_with_DPO.ipynb
with
https://colab.research.google.com/github/CrashingGuru/FGAN-Build-a-thon/blob/main/Notebooks2023/Argilla_Supervised_Finetuning-v1.ipynb
The standalone notebook where we tried argilla trainer is “Crashing and Burning”
So, now, the only way is to replace Argillatrainer with DPO trainer from the new notebook

Description:

This notebook uses the dataset from remote argilla and applies it for DPO Finetuning based on annotated data.

Prerequisites:

the following notebooks are already run:

0. Create the raw dataset in HF hub.

1. Manually create a HF spaces deployment of Argilla

2. Configure the argilla dataset use https://github.com/CrashingGuru/FGAN-Build-a-thon/blob/main/Notebooks2023/1.Argilla_configure_dataset-v1.ipynb

2. add records in the argilla dataset from the raw dataset https://github.com/CrashingGuru/FGAN-Build-a-thon/blob/main/Notebooks2023/2.Read-semi-annotated-push-to-argilla.ipynb

3. annotated the dataset in UI

and finally here we do -- Pull from argilla and apply the annotated data for DPO Finetuning.

**Crash'N'Burn** Ran out of gas (I mean mem). Need to try something else.

In [ ]:
## Install Libraries

Install the latest version of Argilla in Colab, along with other libraries and models used in this notebook.

In [26]:
!pip install -q datasets trl peft

In [27]:
!pip install argilla datasets

In [28]:
!pip install -q bitsandbytes sentencepiece wandb

Prerequisites

Deploy Argilla Server on [HF Spaces](https://huggingface.co/new-space?template=argilla/argilla-template-space).


More info on Installation [here](../getting_started/installation/deployments/deployments.html).

## Secretes needed




* `ARGILLA_API_URL`: It is the url of the Argilla Server.
  * If you're using HF Spaces, it is constructed as `https://[your-owner-name]-[your_space_name].hf.space`.
* `ARGILLA_API_KEY`: It is the API key of the Argilla Server. It is `owner` by default.
* `HF_TOKEN`: It is the Hugging Face API token. It is only needed if you're using a [private HF Space](https://docs.argilla.io/en/latest/getting_started/installation/deployments/huggingface-spaces.html#deploy-argilla-on-spaces). You can configure it in your profile: [Setting > Access Tokens](https://huggingface.co/settings/tokens).
* `workspace`: admin
* wandb : the api key (created free of cost from https://wandb.ai/home


In [29]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
import wandb

%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

/usr/local/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
Note: you may need to restart the kernel to use updated packages.


True

In [30]:
model_name = "mistralai/Mistral-7B-v0.1"
new_model = "ITU-BuildaThon-Mistral-7B"

In [31]:
import argilla as rg
from argilla._constants import DEFAULT_API_KEY

## Access Argilla secret and URL

In [32]:
api_url= os.getenv('my_argilla_url')
api_key= os.getenv('my_argilla_key')

import argilla as rg
rg.init(api_url=api_url, api_key=api_key)

# # If you want to use your private HF Space
# rg.init(extra_headers={"Authorization": f"Bearer {hf_token}"})

## Access HF Login and Token

In [33]:
hf_token = os.getenv('my_hf_token')
from huggingface_hub import login
login(hf_token)

wb_token = os.getenv('my_wandb')
wandb.login(key=wb_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/.netrc


Token is valid (permission: write).
Your token has been saved to /home/.cache/huggingface/token
Login successful


True

## Format dataset

In [34]:
#refer: https://docs.argilla.io/en/latest/tutorials_and_integrations/tutorials/feedback/training-llm-mistral-sft.html
from typing import Dict, Iterator, Any
from argilla.feedback import TrainingTask

dataset = rg.FeedbackDataset.from_argilla(name="fgan-annotate-dataset", workspace="admin")

def formatting_func(sample: Dict[str, Any]) -> Iterator[str]:
    if ANNOTATED_ONLY:
        # Discard if there are no annotations...
        if not sample["quality"]:
            return

        # or if it is annotated as "Bad" or discarded.
        first_annotation = sample["quality"][0]
        if first_annotation["value"] == "Bad" or first_annotation["status"] == "discarded":
            return

    # Filter out responses that are likely low quality
    if len(sample["response"]) <= 2:
        return

    # Add </s><s> between all prompt-response pairs
    prompt = sample["prompt"]
    prompt = prompt.replace("<human>:", f"{tokenizer.eos_token}{tokenizer.bos_token}<human>:")
    prompt = prompt[prompt.find("<human>:"):]
    # Add response and optionally the background to the full text.
    output = prompt + " " + sample["response"]
    if sample["background"]:
        output = sample["background"] + " " + output
    output = output + tokenizer.eos_token
    # We expect one less <s> than </s>, because the Mistral tokenizer will automatically add the BOS
    # at the start of the text when this text is tokenized. When that's done, the format will be exactly
    # what we want
    assert output.count("<s>") + 1 == output.count("</s>")
    return output

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

ANNOTATED_ONLY = False

task = TrainingTask.for_supervised_fine_tuning(formatting_func)
formatted_dataset = dataset.prepare_for_training(framework="trl", task=task)
formatted_dataset



Dataset({
    features: ['id', 'text'],
    num_rows: 98
})

In [37]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 torch.cuda.is_available()                                                                    │
│ ❱ 2 torch.cuda.get_device_name(0)                                                                │
│   3                                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:423 in get_device_name            │
│                                                                                                  │
│    420 │   Returns:                                                                              │
│    421 │   │   str: the name of the device                                                       │
│    422 │   """                                                                                   │
│ ❱  423 │   return get_device_properties(device).name                                             │
│    424                                                                                           │
│    425                                                                                           │
│    426 def get_device_capability(device: Optional[_device_t] = None) -> Tuple[int, int]:         │
│                                                                                                  │
│ /usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:453 in get_device_properties      │
│                                                                                                  │
│    450 │   Returns:                                                                              │
│    451 │   │   _CudaDeviceProperties: the properties of the device                               │
│    452 │   """                                                                                   │
│ ❱  453 │   _lazy_init()  # will define _get_device_properties                                    │
│    454 │   device = _get_device_index(device, optional=True)                                     │
│    455 │   if device < 0 or device >= device_count():                                            │
│    456 │   │   raise AssertionError("Invalid device id")                                         │
│                                                                                                  │
│ /usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:302 in _lazy_init                 │
│                                                                                                  │
│    299 │   │   # are found or any other error occurs                                             │
│    300 │   │   if "CUDA_MODULE_LOADING" not in os.environ:                                       │
│    301 │   │   │   os.environ["CUDA_MODULE_LOADING"] = "LAZY"                                    │
│ ❱  302 │   │   torch._C._cuda_init()                                                             │
│    303 │   │   # Some of the queued calls may reentrantly call _lazy_init();                     │
│    304 │   │   # we need to just return without initializing in that case.                       │
│    305 │   │   # However, we must not let any *other* threads in!                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by 
downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go 
to: https://pytorch.org to insta

## Train model with DPO

In [35]:
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)
model.config.use_cache = False

# Reference model
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
)

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=formatted_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

# Fine-tune model with DPO
dpo_trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 )                                                                                           │
│   10                                                                                             │
│   11 # Model to fine-tune                                                                        │
│ ❱ 12 model = AutoModelForCausalLM.from_pretrained(                                               │
│   13 │   model_name,                                                                             │
│   14 │   torch_dtype=torch.float16,                                                              │
│   15 │   load_in_4bit=True                                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:566 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   563 │   │   │   )                                                                              │
│   564 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   565 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 566 │   │   │   return model_class.from_pretrained(                                            │
│   567 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   568 │   │   │   )                                                                              │
│   569 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.11/site-packages/transformers/modeling_utils.py:3030 in from_pretrained   │
│                                                                                                  │
│   3027 │   │                                                                                     │
│   3028 │   │   if load_in_8bit or load_in_4bit:                                                  │
│   3029 │   │   │   if not torch.cuda.is_available():                                             │
│ ❱ 3030 │   │   │   │   raise RuntimeError("No GPU found. A GPU is needed for quantization.")     │
│   3031 │   │   │   if not (is_accelerate_available() and is_bitsandbytes_available()):           │
│   3032 │   │   │   │   raise ImportError(                                                        │
│   3033 │   │   │   │   │   "Using `load_in_8bit=True` requires Accelerate: `pip install acceler  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: No GPU found. A GPU is needed for quantization.

## Upload model

In [ ]:
# Save artifacts
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

# Flush memory
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()

# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# Push them to the HF Hub
model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

## Inference

In [ ]:
# Format prompt
message = [
    {"role": "system", "content": "You are a helpful assistant chatbot."},
    {"role": "user", "content": "What is a Large Language Model?"}
]
tokenizer = AutoTokenizer.from_pretrained(new_model)
prompt = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)

# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer
)

# Generate text
sequences = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)
print(sequences[0]['generated_text'])

## copied from old notebook

In [ ]:
print(formatted_dataset[80]["text"])

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
data_collator([tokenizer(formatted_dataset[0]["text"])])

In [ ]:
from transformers import DataCollatorForSeq2Seq, BatchEncoding

class DataCollatorForSeq2SeqCopyLabels(DataCollatorForSeq2Seq):
    def __call__(self, features, return_tensors=None) -> BatchEncoding:
        for feature in features:
            if "labels" not in feature:
                feature["labels"] = feature["input_ids"].copy()
        return super().__call__(features, return_tensors=return_tensors)

In [ ]:
data_collator = DataCollatorForSeq2SeqCopyLabels(tokenizer)
data_collator([tokenizer(formatted_dataset[0]["text"])])

In [ ]:
from typing import Optional
import torch
from transformers import TrainerCallback, TrainerControl, TrainerState, GenerationConfig, TrainingArguments, PreTrainedModel, PreTrainedTokenizer


class GenerationCallback(TrainerCallback):
    def __init__(self, prompt: str) -> None:
        super().__init__()
        self.prompt = prompt

    def on_evaluate(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, model: Optional[PreTrainedModel] = None, tokenizer: Optional[PreTrainedTokenizer] = None, **kwargs):
        # Tokenize the prompt and send it to the right device
        inputs = tokenizer(self.prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                generation_config=GenerationConfig(
                    max_new_tokens=50,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                ),
            )
            print(tokenizer.batch_decode(outputs, skip_special_tokens=False)[0])


generation_callback = GenerationCallback("<human>: what are autonomous networks? <bot>:")

In [ ]:
!pip install trl>=0.5.0

In [ ]:
from argilla.feedback import ArgillaTrainer

trainer = ArgillaTrainer(
    dataset=dataset,
    model=model,
    tokenizer=tokenizer,
    task=task,
    framework="trl",
    train_size=0.99,
)

In [ ]:
!pip install peft

In [ ]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
)
trainer.update_config(
    data_collator=data_collator,
    callbacks=[generation_callback],
    peft_config=peft_config,
    max_seq_length=1024,
)

In [ ]:
trainer.update_config(
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    eval_accumulation_steps=16,
    max_steps=3000,
    logging_steps=50,
    learning_rate=5e-5,
    save_strategy="no",
    evaluation_strategy="steps",
    eval_steps=500,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    remove_unused_columns=False,
    fp16=True,
    num_train_epochs=1,
)

In [ ]:
trainer.train("Mistral-7B-v0.1-chat-OIG-3k")